# RDKit Workthrough

This notebook showcases some of the ways I have perviously employed RDKit in my work, using an arbitrary set of compounds as examples (the first 100 molecules listed in PubChem's database). 

My previous work with RDKit involved uncovering relevant features of inter-layer organic molecules that could influence the dimmensionality of periodic structures (primarily perovskites). This work was largely influenced by previous findings from the Wu group at OSU, which showed that four structural features of interlayer amines (steric effect index, eccentricity, ring size, and hydrogen-bond donor count) were able to describe the molecules as either "2D"-forming or "non-2D"-forming. 

For my work, I employed RDKit to tabulate as many features for my set of interlayer organic molecules as possible to assess their potential influence.

In [30]:
import numpy as np
import pandas as pd
import os, re, sys
import matplotlib as plt
from pubchempy import Compound, get_compounds
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, AllChem, Descriptors, Descriptors3D, rdForceFieldHelpers, rdMolDescriptors, rdDistGeom

In [2]:
compounds = [Compound.from_cid(x) for x in range(1, 101)]
smiles = [compounds[x].isomeric_smiles for x in range(0, len(compounds))]
IUPAC_names = [compounds[x].iupac_name for x in range(0, len(compounds))]
data = pd.DataFrame({'Compound': IUPAC_names, 'Compound smiles': smiles})

In [3]:
data

,Compound,Compound smiles
0,3-acetyloxy-4-(trimethylazaniumyl)butanoate,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
1,(2-acetyloxy-3-carboxypropyl)-trimethylazanium,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
2,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ...",C1=CC(C(C(=C1)C(=O)O)O)O
3,1-aminopropan-2-ol,CC(CN)O
4,(3-amino-2-oxopropyl) dihydrogen phosphate,C(C(=O)COP(=O)(O)O)N
...,...,...
95,3-oxobutanoic acid,CC(=O)CC(=O)O
96,"3,5-dihydroxy-2-(hydroxymethyl)-6-[4,5,6-trihy...",C(C1C(C(C(C(O1)O)O)O)OC2C(C(=O)C(C(O2)CO)O)O)O
97,2-oxo-3-sulfanylpropanoic acid,C(C(=O)C(=O)O)S
98,S-[2-[3-[[4-[[[5-(6-aminopurin-9-yl)-4-hydroxy...,CC(=CC(=O)SCCNC(=O)CCNC(=O)C(C(C)(C)COP(=O)(O)...


## Filling out dataframe 

In [4]:
#We will want to begin by including similiar descriptors to those found relevant in previous works
molecules = [Chem.MolFromSmiles(item) for item in smiles]

data['Molecular Weight'] = [Descriptors.MolWt(i) for i in molecules]
data['H-bond Donor Count'] = [Descriptors.NumHDonors(i) for i in molecules]
data['Polar Surface Area'] = [Descriptors.TPSA(i) for i in molecules]
data['Accessible Surface Area'] = [Descriptors.LabuteASA(i) for i in molecules]

In [5]:
data

,Compound,Compound smiles,Molecular Weight,H-bond Donor Count,Polar Surface Area,Accessible Surface Area
0,3-acetyloxy-4-(trimethylazaniumyl)butanoate,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,203.238,0,66.43,83.859416
1,(2-acetyloxy-3-carboxypropyl)-trimethylazanium,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,204.246,1,63.60,83.859416
2,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ...",C1=CC(C(C(=C1)C(=O)O)O)O,156.137,3,77.76,63.088090
3,1-aminopropan-2-ol,CC(CN)O,75.111,2,46.25,31.603260
4,(3-amino-2-oxopropyl) dihydrogen phosphate,C(C(=O)COP(=O)(O)O)N,169.073,3,109.85,57.222171
...,...,...,...,...,...,...
95,3-oxobutanoic acid,CC(=O)CC(=O)O,102.089,1,54.37,40.951355
96,"3,5-dihydroxy-2-(hydroxymethyl)-6-[4,5,6-trihy...",C(C1C(C(C(C(O1)O)O)O)OC2C(C(=O)C(C(O2)CO)O)O)O,340.281,7,186.37,129.803746
97,2-oxo-3-sulfanylpropanoic acid,C(C(=O)C(=O)O)S,120.129,2,54.37,45.741480
98,S-[2-[3-[[4-[[[5-(6-aminopurin-9-yl)-4-hydroxy...,CC(=CC(=O)SCCNC(=O)CCNC(=O)C(C(C)(C)COP(=O)(O)...,849.643,9,363.63,311.188746


# Creating 3D Conformers

In [34]:
m = Chem.MolFromSmiles('C1CCC1OC')
m3=Chem.AddHs(m)
m4 = AllChem.EmbedMolecule(m3, AllChem.ETKDG())
Descriptors3D.Eccentricity(m4)

ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.CalcEccentricity(int)
did not match C++ signature:
    CalcEccentricity(RDKit::ROMol mol, int confId=-1, bool useAtomicMasses=True, bool force=True)

In [27]:
smi_to_descriptors('CC(=O)CC(=O)O')

array([1.02031694e+02, 3.00000000e+00, 1.00000000e+00, 2.00000000e+00,
       1.00000000e+00, 2.00000000e+00, 3.00000000e+00, 0.00000000e+00,
       5.00000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.09513553e+01, 5.43700000e+01, 5.01000000e-02,
       2.29338000e+01])

In [19]:
Molecules3D = []
for molecule in molecules:
    #molecule = Chem.MolToMolBlock(molecule)
    hydrogenated = Chem.AddHs(molecule)
    #hydrogenated = Chem.MolToMolBlock(hydrogenated)
    embeded = AllChem.EmbedMolecule(hydrogenated)
    loop_3D = Chem.rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(hydrogenated)
    Molecules3D.append(loop_3D)

for item in Molecules3D:
    print(item)

[(0, -57.63207331052396)]
[(0, -14.80328480520914)]
[(0, -2.5249733400358974)]
[(0, 19.81610186764315)]
[(0, -162.54807306967584)]
[(0, 59.235109232869114)]
[(0, -3.0545813455464126)]
[(0, 45.53670269858786)]
[(0, -100.64976450500416)]
[(1, -13.241234266589633)]
[(0, 6.251242819449175)]
[(0, 2.3884909746765093)]
[(0, 15.894480868067312)]
[(1, 14.067090746459321)]
[(0, 79.96158386645705)]
[(0, -29.38213060247226)]
[(0, 78.41897411745767)]
[(0, 83.15005227616587)]
[(0, 11.853453736919192)]
[(0, -20.162517874191145)]
[(0, 31.49035093773021)]
[(0, 31.473359258768646)]
[(0, 19.97972133956938)]
[(1, 90.7582269484254)]
[(0, 64.48013744724929)]
[(1, 11.320212736204617)]
[(1, -524.9639949479517)]
[(0, -0.9450193191812453)]
[(0, 46.72005233664227)]
[(0, 24.947737871815924)]
[(0, -134.45104721341824)]
[(0, -63.394107181614686)]
[(0, 11.509764091059914)]
[(0, 12.39223362405971)]
[(0, -61.67733136863443)]
[(0, -18.214638517834814)]
[(0, 57.28923785611194)]
[(0, 29.767962610398467)]
[(0, 39.41055393

In [ ]:
#Visualizing structures
test_mol = Chem.MolFromSmiles(compounds_smiles[2])
pic = AllChem.Compute2DCoords(test_mol)

In [ ]:
Draw.MolToImage(test_mol)

In [ ]:
#molecules = [Chem.MolFromSmiles(compounds_smiles[x] for x in range(0, 20))]
molecule= []
for x in range(0, 20):
    loop_molecule = Chem.MolFromSmiles(compounds_smiles[x])
    molecule.append(loop_molecule)    

In [ ]:
img=Draw.MolsToGridImage(molecule,molsPerRow=2,subImgSize=(400,150),legends=[compounds_names[x] for x in range(0, 20)])
img

In [ ]:
def find_nearest_match(smiles, greatest_similarity = 0, nearest_match = 'None'):
    search_mol = Chem.MolFromSmiles(smiles)
    search_fingerprint = Chem.RDKFingerprint(search_mol)
    #greatest_similarity = 0
    #nearest_match = 'None'
    for item in compounds_smiles:
        item_as_mol = Chem.MolFromSmiles(item)
        item_as_fingerprint = Chem.RDKFingerprint(item_as_mol)
        similarity = DataStructs.FingerprintSimilarity(search_fingerprint,item_as_fingerprint)
        if similarity > greatest_similarity:
            greatest_similarity = round(similarity, 3)
            nearest_match = item
    return_phrase = "The nearest match is " + str(nearest_match) + " with a Tanimoto similarity of " + str(greatest_similarity)
    print(return_phrase)

In [ ]:
find_nearest_match('CCCCCCO')

In [ ]:
class Molecule:
    def __init__(self, smiles):
        self.smiles = smiles
        molecule = Chem.MolFromSmiles(smiles)
        fingerprint = Chem.RDKFingerprint(molecule)
        
    def 